In [1]:
import datetime
import numpy
import netCDF4
from osgeo import gdal, osr
import ogr
import os
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing as mp
import collections
import logging
import time
import struct

logging.basicConfig(level=logging.DEBUG, filename=r'P:\projects\SILO\siloLog.txt')

#example
#start = '2015-01-01'
#end = '2015-03-23'
#sd = dt.strptime(start,'%Y-%m-%d')
#ed = dt.strptime(end,'%Y-%m-%d')
#delta = ed - sd
#for i in range(delta.days+1):
#    dict[sd + td(days=i)] = <something>

reqStart = '2019-01-01'
#reqEnd = '1981-12-31'
#reqEnd = '1980-12-31'
reqEnd = '2019-10-22'

"""
inputGeoCoordsFile = r'P:\projects\SILO\Wes\MNDB_geocoords.csv'
outputDir = r'P:\projects\SILO\Wes\testOut2'
siloNetCDFDir = r'P:\projects\SILO\Wes\mwet'

varNameInSilo = 'et_morton_wet'
varForColName = 'pet'

"""
inputGeoCoordsFile = r'C:\DDrive\Maria\CatchModCSVs\WT_RC8_30m_climcoords.csv'
outputDir = r'C:\DDrive\Maria\ClimateOuts'
#siloNetCDFDir = r'C:\DDrive\Maria\netCDFs\rainfall'
#siloNetCDFDir = r'C:\DDrive\Maria\netCDFs\rainfall'
siloNetCDFDir = r'\\nrm02471\Climate_extension\netCDF_dumps\Downloaded_Oct2019\rainfall'

varNameInSilo = 'daily_rain'
varForColName = 'rainfall'



#Col names for GeoCoords CSV
catGrpName = 'CATCHGROUP'
latName = 'SILO_LAT'
longName = 'SILO_LONG'
countName = 'COUNT'

In [2]:
#Base output dir
if not os.path.exists(outputDir):
    os.makedirs(outputDir)
    print("Made dir: " + outputDir)


In [3]:
#Read in GeoCoords
GeoCoordsVals = {}

geoCoordsDF = pd.read_csv(inputGeoCoordsFile)
#for index, row in geoCoordsDF.iterrows():
#    if not row[catGrpName] in GeoCoordsVals:
#        #print('Adding: ' + str(row[catFileRasID]) + ' and ' + str(row[catFileCatName]))
#        GeoCoordsVals[row[catGrpName]] = row[catFileCatName]

allCats = geoCoordsDF[catGrpName].unique().tolist()
colNames = [varForColName + ' for ' + cat for cat in allCats]
print(len(allCats))

517


In [4]:

sd = datetime.datetime.strptime(reqStart,'%Y-%m-%d')
ed = datetime.datetime.strptime(reqEnd,'%Y-%m-%d')
##delta = ed - sd

sYear = sd.year
eYear = ed.year

allYears = []
for y in range(sYear, eYear + 1):
    if not y in allYears:
        allYears.append(y)

#date_today = datetime.now()
#days = pd.date_range(date_today, date_today + timedelta(7), freq='D')
date_rng = pd.date_range(sd, ed, freq='D')
#print(str(type(days[0])))
daysAsList = date_rng.tolist()
#mainDF = pd.DataFrame({'Date': days})
mainDF = pd.DataFrame(date_rng, columns=['tempDate'])
#mainDF = mainDF.set_index('Date')

#Getting date as index????
mainDF['Date'] = pd.to_datetime(mainDF['tempDate'])
mainDF = mainDF.set_index('Date')
mainDF.drop(['tempDate'], axis=1, inplace=True)


for col in colNames:
    mainDF[col] = ''

resultsDict = {}
#for d in daysAsList:
#    resultsDict[d] = {}



mainDF.head()
#print(mainDF)

,rainfall for SC #4,rainfall for SC #35,rainfall for SC #1,rainfall for SC #5,rainfall for SC #34,rainfall for SC #90488,rainfall for SC #2,rainfall for SC #3,rainfall for SC #9,rainfall for SC #33,...,rainfall for SC #381,rainfall for SC #379,rainfall for SC #380,rainfall for SC #469,rainfall for SC #376,rainfall for SC #386,rainfall for SC #377,rainfall for SC #378,rainfall for SC #387,rainfall for SC #388
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,,,,,,,,,,,...,,,,,,,,,,
2019-01-02,,,,,,,,,,,...,,,,,,,,,,
2019-01-03,,,,,,,,,,,...,,,,,,,,,,
2019-01-04,,,,,,,,,,,...,,,,,,,,,,
2019-01-05,,,,,,,,,,,...,,,,,,,,,,


In [5]:
#class to contain a raster band
DailyRasterBand = collections.namedtuple('DailyRasterBand', [
    'dateOfRaster',
    'RasterBand',
    'RowCount',
    'ColCount',
    'geoTransObject',
    'scaleFactor',
    'noDataVal'
])

In [6]:
def processBand(number, allBands):

    result = {}
    srcband = allBands[number]

    #logging.info('Checking this item: ' + str(number) + ' is a ' + str(type(srcband.RasterBand))
    #             + 'of date ' + srcband.dateOfRaster)
   
    #scaleFactor = 1.0
        
    #sf = srcband.RasterBand.GetMetadataItem(varNameInSilo + '_scale_factor')
    #nd = srcband.RasterBand.GetMetadataItem(varNameInSilo + '__FillValue')
    #nd2 = srcband.RasterBand.GetNoDataValue()

        
    scaleFactor = float(srcband.scaleFactor)
    rawNoData = float(srcband.noDataVal)
    
    fakeSiloGT = srcband.geoTransObject
    #siloArray = srcband.RasterBand.ReadAsArray()
    ##just make sure there's no gaps? Not necessary?
    ##siloMask = numpy.ma.masked_where(siloArray == nd, siloArray)
    
    #print(srcband.dateOfRaster + ' ' + str(srcband.RowCount))
    
    #logging.info('Checking this array: ' + str(siloArray))
        
    #print(dateForFileName + ': ' + str(siloMask[500,500] * scaleFactor))
    
    ##NOTE: Updated GDAL means we don't need to invert anymore
    #Invert Y-axis values...
    #Really important
    #invertArray = srcband.RasterArray[::-1,:]
        
    for thisCat in allCats:
            
        colName = varForColName + ' for ' + thisCat
        catTodaySum = 0.0
        catTodayCount = 0.0
            
        catDF = geoCoordsDF.loc[geoCoordsDF[catGrpName] == thisCat]
            
        for index, row in catDF.iterrows():
                
            longVal = float(row[longName])
            latVal = float(row[latName])
            countVal = int(row[countName])
                
            geoXcell = int((longVal - fakeSiloGT[0]) / fakeSiloGT[1]) #x pixel
            geoYcell = int((latVal - fakeSiloGT[3]) / fakeSiloGT[5]) #y pixel
                
            ##NOTE: Updated GDAL means we don't need to invert anymore
            #catTodaySum += (invertArray[geoYcell,geoXcell] * scaleFactor) * countVal
            #catTodaySum += (srcband.RasterArray[geoYcell,geoXcell] * scaleFactor) * countVal
            rawval = srcband.RasterBand.ReadRaster(geoXcell,geoYcell,1,1,buf_type=gdal.GDT_Float32)
            #intval = struct.unpack('b' , rawval)
            intval = struct.unpack('f', rawval)
            
            #Ignore NoData
            if (intval[0] == rawNoData):
                continue
                """
                print("rawval = " + str(rawval))
                print("longVal = " + str(longVal))
                print("geoXcell = " + str(geoXcell))
                print("latVal = " + str(latVal))
                print("geoYcell = " + str(geoYcell))
                print("intval = " + str(intval[0]))
                """
            
            #print("rawval = " + str(rawval))
            #print("geoXcell = " + str(geoXcell))
            #print("geoYcell = " + str(geoYcell))
            #print("intval = " + str(intval[0]))
            catTodaySum += (intval[0] * scaleFactor) * countVal
            catTodayCount += countVal
        
        
        #Calc average
        avVal = 0
        if catTodayCount > 0:
            avVal = catTodaySum / catTodayCount
            
        result.update({thisCat:avVal})
    
    #Try and jam into dataframe
    #for cat,val in result.items():
    #    colName = varForColName + ' for ' + cat
    #    mainDF[srcband.dateOfRaster][colName] = val
    
    resultsDict[srcband.dateOfRaster] = result
    
    return
    #return {
    #    'name': item.name,
    #    'age': 2017 - item.born
    #}

In [7]:
#load the data
#print(allYears)
print('started at: ' + str(datetime.datetime.now()))
data = (['a', '2'], ['b', '4'], ['c', '6'], ['d', '8'],['e', '1'], ['f', '3'], ['g', '5'], ['h', '7'])
numbers = [5, 10, 15, 20, 25]
procs = []

for thisYear in allYears:
    
    siloFile = os.path.join(siloNetCDFDir, str(thisYear) + '.' + varNameInSilo + '.nc')
    print('Processing: ' + siloFile)
    #print(type(thisYear))
    
    #Need the gdal version of silo to give us a few params, but will close of netCDF version pronto...
    siloDS = gdal.Open(siloFile)
    
    #Updated GDAL means we don't need this anymore
    """
    ##Need this to calc pixel size as well as work out order of y-axis
    netCDFData = netCDF4.Dataset(siloFile, 'r')

    #Grab the info we need to build GeoTransform object
    #Checking for order of y-axis coordinates, would be needed if writing raster form Numpy array
    
    lats = netCDFData.variables['lat']
    lons = netCDFData.variables['lon']
    times = netCDFData.variables['time']
    
    first_lat = lats[:1]
    last_lat = lats[len(lats)-1:]
    #If first_lat is LESS THAN last_lat, numpy array would be written to raster upside down...
    
    first_lon = lons[:1]
    last_lon = lons[len(lons)-1:]
    
    colCount = siloDS.RasterXSize
    rowCount = siloDS.RasterYSize
    
    pixWidth = (last_lon - first_lon) / (siloDS.RasterXSize - 1)
    pixHeight = (last_lat - first_lat) / (siloDS.RasterYSize - 1)

    if first_lat < last_lat:
        #adjust for our use
        pixHeight *= -1
    
    
    siloLeft = min(lons) - (0.5 * pixWidth)#adjusting to get cell edge, not centre
    siloTop = max(lats) - (0.5 * pixHeight)#adjusting to get cell edge, not centre
    
    netCDFData.close()
    """
    
    #fakeSiloGT = [siloLeft, pixWidth, 0, siloTop, 0, pixHeight]
    #The GDal dataset should have the pixel height etc already, maybe my old GDAL version couldn't read it???
    fakeSiloGT = siloDS.GetGeoTransform()
    
    #Jan 1
    siloDStart = datetime.datetime(year=thisYear, month=1, day=1)
    
    theRowCount = siloDS.RasterYSize
    theColCount = siloDS.RasterXSize
    
    #theDataSet.RasterCount is the number of days we have
    #remembering that 'range' give a STOP value that isn't included
    #however, depsite 'band' starting at zero, band IDs will start at 1!
    
    allbands = []
    for band in range(0, siloDS.RasterCount):
        
        dateNow = siloDStart + datetime.timedelta(days=band)
        dateForFileName = str(dateNow.year) + str(dateNow.month).zfill(2) + str(dateNow.day).zfill(2)
        dateTimeStampKey = str(dateNow.year) + '-' + str(dateNow.month).zfill(2) + '-' + str(dateNow.day).zfill(2)
        
        if not dateNow in daysAsList:
            continue
        
        #print("Getting: " + dateTimeStampKey + " from Idx: " + str(band))
        theBand = siloDS.GetRasterBand(band + 1)
        
        #Updated GDAL means we don't need var name
        #theSF = theBand.GetMetadataItem(varNameInSilo + '_scale_factor')
        #theND = theBand.GetMetadataItem(varNameInSilo + '__FillValue')
        theSF = theBand.GetMetadataItem('scale_factor')
        theND = theBand.GetMetadataItem('_FillValue')
        
        scaleFactor = float(theSF)
        rawNoData = float(theND)

        
        allbands.append(DailyRasterBand(dateOfRaster=dateTimeStampKey,
                                        RasterBand=theBand, RowCount=theRowCount, ColCount=theColCount,
                                        geoTransObject=fakeSiloGT, scaleFactor=scaleFactor, noDataVal=rawNoData))
    
    num_cores = mp.cpu_count()
    procList = list(range(1, siloDS.RasterCount + 1))
    #Parallel(n_jobs=num_cores)(delayed(doubler) for i in numbers)#in enumerate(numbers)#, siloDS=siloDS
    #Parallel(n_jobs=num_cores, require='sharedmem')(delayed(doubler)(i) for i in range(5))
    Parallel(n_jobs=num_cores, require='sharedmem')(delayed(processBand)(i, allbands) for i in range(0, len(allbands)))
    #print(ff)
    #print(shared_set)
    
    #pool = mp.Pool(mp.cpu_count())
    #print(str(mp.cpu_count()))
    ##results = [pool.apply(grabDailyVals2, args=(row)) for row in procList]
    #pool.close() 
    
#mainDF.head()
print('finished at: ' + str(datetime.datetime.now()))

started at: 2019-11-13 09:48:08.490369
Processing: \\nrm02471\Climate_extension\netCDF_dumps\Downloaded_Oct2019\rainfall\2019.daily_rain.nc
finished at: 2019-11-13 09:54:32.990889


In [8]:
for t,data in resultsDict.items():
    for col,val in data.items():
        colName = varForColName + ' for ' + col
        mainDF.loc[t][colName] = val

In [9]:
mainDF.head()

,rainfall for SC #4,rainfall for SC #35,rainfall for SC #1,rainfall for SC #5,rainfall for SC #34,rainfall for SC #90488,rainfall for SC #2,rainfall for SC #3,rainfall for SC #9,rainfall for SC #33,...,rainfall for SC #381,rainfall for SC #379,rainfall for SC #380,rainfall for SC #469,rainfall for SC #376,rainfall for SC #386,rainfall for SC #377,rainfall for SC #378,rainfall for SC #387,rainfall for SC #388
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,40.2209,33.9903,39.7347,38.0331,28.2297,39.3871,40.0065,39.1984,32.5248,27.1172,...,0.191897,0.154286,0.197584,0.219111,0.0979917,0.24508,0.297752,0.1,0.13571,0.122408
2019-01-02,17.6677,14.5951,19.1671,17.4718,12.3062,21.5231,20.3207,18.9947,14.9778,12.3119,...,0.291897,0.619655,0.571933,0.991054,0.593975,1.0337,1.40431,0.6,1.08393,1.24109
2019-01-03,0.619096,0.214109,1.2,0.805815,0.0049081,2.31041,1.56519,1.19836,0.43856,0.0613439,...,0,0,0,0,0,0,0,0,0,0
2019-01-04,0.409819,1.3427,0.1,0.529781,2.41321,0.0348825,0.141558,0.101228,1.18709,2.20943,...,0,0,0,0,0,0,0,0,0,0
2019-01-05,5.22836,3.85225,7.89871,6.67431,3.47098,15.3447,10.8203,8.39263,5.05249,3.87324,...,0.191897,0.299009,0.334758,0.430126,0.195983,0.48657,0.497018,0.2,0.310704,0.203966


In [10]:
mainDF.to_csv(r'C:\DDrive\Maria\WTrain1980.csv', index=True)